In [10]:
import pandas as pd
import re

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import SimpleSequentialChain,SequentialChain
from langchain.chains.llm import LLMChain
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import os
from graphdatascience import GraphDataScience
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv("C:/Users/MBBLABS/TH THNM/Langchain_results.csv")

In [3]:
df.drop(["Unnamed: 0"],axis=1,inplace=True)

In [4]:
df.head()

,V_BIOGRAPHY_repo,Crime_extracted,Date_of_crime_extracted,Material Crime flag,Crime entity using NER ( pre-trained model),Latest date of crime ( using NER ),With in 7 years flag,RFI/THNM,Unnamed: 9,Unnamed: 10
0,mar 2022 - no further information reported. [...,"The answer is ""no further information reported...","The answer is ""no further information reported...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,dec 2019 - pleaded not guilty to the criminal...,Bribery,Between jul 2019 - oct 2019,material,Bribery,Jul-19,yes,RFI,NaN,NaN
2,dec 2018 - reportedly detained. sentenced to ...,The individual is involved in terror-related c...,"The answer to the question ""When did the terro...",material,terror-related crime,Dec-18,yes,RFI,NaN,NaN
3,jun 2018 - declared as proclaimed offender by...,Involved in organized crime.,\nOrganized crime occurred in June 2018.,??,organized crime,Jun-18,NaN,NaN,NaN,NaN
4,aug 2013 - declared as proclaimed offender by...,Theft,Theft occurred in Aug 2013.,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df["Crime_extracted"]=df["Crime_extracted"].apply(lambda x: "" if pd.isna(x) else x)
df["Crime_extracted"]=df["Crime_extracted"].apply(lambda x: re.sub("\n","",x))
df["Date_of_crime_extracted"]=df["Date_of_crime_extracted"].apply(lambda x: "" if pd.isna(x) else x)
df["Date_of_crime_extracted"]=df["Date_of_crime_extracted"].apply(lambda x: re.sub("\n","",x))

In [20]:
# !pip install gliner

In [6]:
from gliner import GLiNER

In [ ]:
model = GLiNER.from_pretrained("urchade/gliner_mediumv2.1")
model.eval()
print("ok")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

D:\anaconda\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\anaconda\Lib\site-packages\transformers\convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
df["Crime_extracted"].loc[1]
df["Crime_entity_Gliner"]=""

In [ ]:
text = df["Crime_extracted"].loc[1]

labels = ["crime"]

entities = model.predict_entities(text, labels, threshold=0.1)
import numpy as np
for entity in entities:
    print(entity["text"], "=>", entity["label"])

In [ ]:

import warnings
warnings.filterwarnings("ignore")
for i,text in enumerate(df.loc[:100,"Crime_extracted"]):
  if pd.notna(text):
      entities = model.predict_entities(text, labels, threshold=0.4)
      # print(entities)
      if len(entities)>0 :
           df["Crime_entity_Gliner"].loc[i]= entities[0]["text"]
      else:
           df["Crime_entity_Gliner"].loc[i]=""
  else:
       df["Crime_entity_Gliner"].loc[i]="no crime extracted"

In [ ]:
df["Date_of_crime_entity_Gliner"]

In [82]:
# text = df["Date_of_crime_extracted"].loc[1]

labels = ["Date_of_crime_entity_Gliner"]

# entities = model.predict_entities(text, labels, threshold=0.4)

# for entity in entities:
#     print(entity["text"], "=>", entity["label"])

import numpy as np
# for entity in entities:
#     print(entity["text"], "=>", entity["label"])
import warnings
warnings.filterwarnings("ignore")
for i,text in enumerate(df.loc[:100,"Date_of_crime_extracted"]):
  if pd.notna(text):
      entities = model.predict_entities(text, labels, threshold=0.4)
      # print(entities)
      if len(entities)>0 :
           df["Date_of_crime_entity_Gliner"].loc[i]= entities[0]["text"]
      else:
           df["Date_of_crime_entity_Gliner"].loc[i]=""
  else:
       df["Date_of_crime_entity_Gliner"].loc[i]="no date of crime extracted"

In [86]:
pd.set_option("display.max_rows",None)
df[["V_BIOGRAPHY_repo"	,"Crime_extracted",	"Date_of_crime_extracted","Crime entity using NER ( pre-trained model)","Latest date of crime ( using NER )"]].head(100)

,V_BIOGRAPHY_repo,Crime_extracted,Date_of_crime_extracted,Crime entity using NER ( pre-trained model),Latest date of crime ( using NER )
0,mar 2022 - no further information reported. [...,"The answer is ""no further information reported...","The answer is ""no further information reported...",,
1,dec 2019 - pleaded not guilty to the criminal...,Bribery,Between jul 2019 - oct 2019,,jul 2019
2,dec 2018 - reportedly detained. sentenced to ...,The individual is involved in terror-related c...,"The answer to the question ""When did the terro...",terror-related crimes,apr 2022
3,jun 2018 - declared as proclaimed offender by...,Involved in organized crime.,Organized crime occurred in June 2018.,organized crime,June 2018
4,aug 2013 - declared as proclaimed offender by...,Theft,Theft occurred in Aug 2013.,Theft,Aug 2013
5,dec 2011 - held responsible for misrepresenta...,"In the provided context, the individual is inv...",In the context: DEC 2011,Misrepresentation in Business Records,DEC 2011
6,jan 2009 - sentenced by cbi to 5 years rigoro...,"The answer is ""fraud""",Jan-09,fraud,Jan-09
7,aug 2021 - first information report filed by ...,The individual is involved in the crime of att...,Attempted murder committed in Aug 2021.,attempt to murder,Aug 2021
8,oct 2018 - wanted by hungarian police for fai...,The individual is wanted by the Hungarian poli...,The answer can be found in the context provide...,failure to pay child support,oct 2018
9,2009 - reportedly declared as proclaimed offe...,"The individual is involved in ""dacoity"" in few...",In 2009.,dacoity,


### Langchain Ollama Chatmodel ,Zero shot fine tuning

In [4]:
%%time
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(temperature=0, model = "llama3")

CPU times: total: 250 ms
Wall time: 328 ms


In [74]:
pd.set_option("display.max_colwidth",None)
# df["Crime_date_combined"].head()

### using on only langchain llms with chat prompt

In [79]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser


df["crime_entity"]=""
df["date_entity"]=""
system_prompt = """
You are a helpful, respectful and honest assistant. Always provide accurate answer based on the context given. 
 please identify the entities present in the context. please tell me the crime type and date if it is present. else dont provide any information. 
 provide in json format having keys as crime and date.If not able to extract both entities, then provide json with same keys but empty string as values.
 Please do not provide any additional text apart from json"""
crime_schema = ResponseSchema(name="crime",
                                   description="This refers to crime entity present in given text.")

date_schema = ResponseSchema(name="date",
                                       description="This is refers to date present in given text.")


response_schema = [
    crime_schema ,
    date_schema,
]

# prompt_template = ChatPromptTemplate.from_template(template)
# messages = prompt_template.format_messages(text=text, format_instructions=format_instructions)

output_parser = StructuredOutputParser.from_response_schemas(response_schema)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)
df["time_of_entity_extr"]=0

df["Crime_date_combined"]=df["Crime_extracted"]+" "+df["Date_of_crime_extracted"]
texts=df["Crime_date_combined"].loc[:5]

for i,text in zip(range(len(texts)),texts):
    print(text)
# text = """ Jun 2021 - reportedly sentenced to 10 years imprisonment by Muse District Court for human trafficking. Mar 2023 - no further information reported."""
# df.loc[:1,"Crime_extracted"]
    prompt_template_entity = ChatPromptTemplate.from_messages([ ("system", f'''{system_prompt}'''), ("human", "{user_input}"),])
    prompt_template_entity.input_variables
    start=time.time()
    final_prompt=prompt_template_entity.format_prompt(user_input=text).to_messages()
    output=llm(final_prompt)
    end=time.time()
    
    df["time_of_entity_extr"]=end-start
    output_dict=  output_parser.parse(output.content)
    df["crime_entity"].loc[i]=output_dict["crime"]
    df["date_entity"].loc[i]=output_dict["date"]

    print(output.content)



The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"crime": string  // This refers to crime entity present in given text.
	"date": string  // This is refers to date present in given text.
}
```
The answer is "no further information reported" as it is contained within the context. The answer is "no further information reported" in the context.


C:\Users\MBBLABS\AppData\Local\Temp\ipykernel_64240\2312110005.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["crime_entity"].loc[i]=output_dict["crime"]
C:\Users\MBBLABS\AppData\Local\Temp\ipykernel_64240\2312110005.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date_entity"].loc[i]=output_dict["date"]


{"crime": "", "date": ""}
Bribery Between jul 2019 - oct 2019


C:\Users\MBBLABS\AppData\Local\Temp\ipykernel_64240\2312110005.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["crime_entity"].loc[i]=output_dict["crime"]
C:\Users\MBBLABS\AppData\Local\Temp\ipykernel_64240\2312110005.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date_entity"].loc[i]=output_dict["date"]


{"crime": "Bribery", "date": "jul 2019 - oct 2019"}
The individual is involved in terror-related crimes. The answer to the question "When did the terrorist act take place?" is:

apr 2022


...

C:\Users\MBBLABS\AppData\Local\Temp\ipykernel_64240\2312110005.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["crime_entity"].loc[i]=output_dict["crime"]
C:\Users\MBBLABS\AppData\Local\Temp\ipykernel_64240\2312110005.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date_entity"].loc[i]=output_dict["date"]


{"crime": "organized crime", "date": "June 2018"}
Theft Theft occurred in Aug 2013.


C:\Users\MBBLABS\AppData\Local\Temp\ipykernel_64240\2312110005.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["crime_entity"].loc[i]=output_dict["crime"]
C:\Users\MBBLABS\AppData\Local\Temp\ipykernel_64240\2312110005.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date_entity"].loc[i]=output_dict["date"]


{"crime": "Theft", "date": "Aug 2013"}
In the provided context, the individual is involved in the crime of "Misrepresentation in Business Records" in Dec 2011. In the context: DEC 2011
{"crime": "Misrepresentation in Business Records", "date": "DEC 2011"}


C:\Users\MBBLABS\AppData\Local\Temp\ipykernel_64240\2312110005.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["crime_entity"].loc[i]=output_dict["crime"]
C:\Users\MBBLABS\AppData\Local\Temp\ipykernel_64240\2312110005.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date_entity"].loc[i]=output_dict["date"]


### using actual langchain with prompt

In [87]:


from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser


df["crime_entity"]=""
df["date_entity"]=""
system_prompt = """
You are a helpful, respectful and honest assistant. Always provide accurate answer based on the context given. 
 please identify the entities present in the context. please tell me the crime type and date if it is present. else dont provide any information. 
 provide in json format having keys as crime and date.If not able to extract both entities, then provide json with same keys but empty string as values.
 Please do not provide any additional text apart from json"""
crime_schema = ResponseSchema(name="crime",
                                   description="This refers to crime entity present in given text.")

date_schema = ResponseSchema(name="date",
                                       description="This is refers to date present in given text.")


response_schema = [
    crime_schema ,
    date_schema,
]

# prompt_template = ChatPromptTemplate.from_template(template)
# messages = prompt_template.format_messages(text=text, format_instructions=format_instructions)

output_parser = StructuredOutputParser.from_response_schemas(response_schema)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)
df["time_of_entity_extr"]=0

df["Crime_date_combined"]=df["Crime_extracted"]+" "+df["Date_of_crime_extracted"]
texts=df["Crime_date_combined"].loc[:5]

for i,text in zip(range(len(texts)),texts):
    print(text)
# text = """ Jun 2021 - reportedly sentenced to 10 years imprisonment by Muse District Court for human trafficking. Mar 2023 - no further information reported."""
# df.loc[:1,"Crime_extracted"]
    prompt_template_entity = ChatPromptTemplate.from_messages([ ("system", f'''{system_prompt}'''), ("human", "{user_input}"),])
    prompt_template_entity.input_variables
    start=time.time()
    # final_prompt=prompt_template_entity.format_prompt(user_input=text).to_messages()
    # output=llm(final_prompt)
    llm_chain = LLMChain(llm=llm, prompt=prompt_template_entity, verbose=True)
    output = llm_chain.run(user_input= text)
    end=time.time()
    
    df["time_of_entity_extr"]=end-start
    print(output)
    output_dict=  output_parser.parse(output)
    df["crime_entity"].loc[i]=output_dict["crime"]
    df["date_entity"].loc[i]=output_dict["date"]

    # print(output)






The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"crime": string  // This refers to crime entity present in given text.
	"date": string  // This is refers to date present in given text.
}
```
The answer is "no further information reported" as it is contained within the context. The answer is "no further information reported" in the context.


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful, respectful and honest assistant. Always provide accurate answer based on the context given. 
 please identify the entities present in the context. please tell me the crime type and date if it is present. else dont provide any information. 
 provide in json format having keys as crime and date.If not able to extract both entities, then provide json with same keys but empty string as values.
 Please do not provide any additional text apart from json
Human: The answer 

In [88]:
df

,V_BIOGRAPHY_repo,Crime_extracted,Date_of_crime_extracted,Material Crime flag,Crime entity using NER ( pre-trained model),Latest date of crime ( using NER ),With in 7 years flag,RFI/THNM,Unnamed: 9,Unnamed: 10,Crime_date_combined,crime_entity,date_entity,time_of_entity_extr
0,"mar 2022 - no further information reported. [note] this record supports the inclusion criteria for individuals connected, associated, affiliated, linked to, or who own or control a sanctioned entity (ofac sanctions list).","The answer is ""no further information reported"" as it is contained within the context.","The answer is ""no further information reported"" in the context.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"The answer is ""no further information reported"" as it is contained within the context. The answer is ""no further information reported"" in the context.",,,9.99959
1,"dec 2019 - pleaded not guilty to the criminal charges of bribery (myr295,000) (jul 2019 - oct 2019). bail allowed. case mention scheduled for jan 2020. aug 2021 - ordered to enter defence. case mention scheduled for oct 2021. mar 2022 - convicted by the sessions court on criminal charges of bribery. sentenced to 58 years imprisonment and fined myr1.05m in default 72 months imprisonment. allowed stay of execution of the sentence pending appeal to the high court.",Bribery,Between jul 2019 - oct 2019,material,Bribery,Jul-19,yes,RFI,NaN,NaN,Bribery Between jul 2019 - oct 2019,Bribery,jul 2019 - oct 2019,9.99959
2,"dec 2018 - reportedly detained. sentenced to death for killing law enforcement agents and destruction of government educational institutions. apr 2022 - no further information reported.[crime - terror category notice] this category includes information about individuals who have reportedly been convicted of a terror related crime. inclusion in the category does not mean that an individual or entity is a terrorist or terrorist organisation. individuals may appeal their conviction; you should review the content carefully and in accordance with our terms and conditions, further enquiries should be made of the report subject to investigate the conviction or the outcome of any appeal and whether the allegations of wrongdoing are denied.",The individual is involved in terror-related crimes.,"The answer to the question ""When did the terrorist act take place?"" is:\n\napr 2022",material,terror-related crime,Dec-18,yes,RFI,NaN,NaN,"The individual is involved in terror-related crimes. The answer to the question ""When did the terrorist act take place?"" is:\n\napr 2022",terror-related crimes,Apr 2022,9.99959
3,jun 2018 - declared as proclaimed offender by khairpur court for involvement in organised crime related activities. may 2022 - no further information reported.,Involved in organized crime.,\nOrganized crime occurred in June 2018.,??,organized crime,Jun-18,NaN,NaN,NaN,NaN,Involved in organized crime. \nOrganized crime occurred in June 2018.,organized crime,June 2018,9.99959
4,"aug 2013 - declared as proclaimed offender by karachi court for alleged involvement in theft, arms and ammunition. jun 2022 - no further information reported.",Theft,Theft occurred in Aug 2013.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Theft Theft occurred in Aug 2013.,Theft,Aug 2013,9.99959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5744,"feb 2011 - fined inr200,000 by sebi for fraudulent and unfair trade practices in the matter of tulip it services limited. aug 2023 - no further information reported.",NaN,Bribery occurred in July 2019 - October 2019.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,9.99959
5745,jul 2010 - sentenced by longhua district court to 7 years imprisonment for participation in an organised crime group and illegal business operation (2000-2009). appealed sentence. feb 2011 - sentence upheld by haikou municipal intermediate court. may 2023 - no further information reported.,NaN,Bribery occurred in July 2019 - October 2019.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,9.99959
5746,"aug 2023 - fined inr500,000 

In [71]:
# df["Crime_date_combined"]=df["Crime_extracted"]+" "+df["Date_of_crime_extracted"]
# text=df["Crime_date_combined"].loc[1:10]

# # text = """ Jun 2021 - reportedly sentenced to 10 years imprisonment by Muse District Court for human trafficking. Mar 2023 - no further information reported."""
# # df.loc[:1,"Crime_extracted"]

# prompt_template_entity.input_variables
# start=time.time()
# final_prompt=prompt_template_entity.format_prompt(user_input=text).to_messages()
# output=llm(final_prompt)
# end=time.time()
# print(output.content)

In [10]:
# system_prompt = """
# You are a helpful, respectful and honest assistant. Always provide accurate answer based on the context given. 
#  please identify the entities present in the context. please tell me the crime type and reported date as well if it is present. else dont provide any information. provide response in json format"""

# prompt_template_entity = ChatPromptTemplate.from_messages([ ("system", f'''{system_prompt}'''), ("human", "{user_input}"),])
# for i,text in enumerate(df.loc[:1,"Crime_extracted"]):
#     print(text)
#     llm_chain = LLMChain(llm=llm, prompt=prompt_template_entity, verbose=True)
#     output = llm_chain.run(user_input= text)
#     print(output)

The answer is "no further information reported" as it is contained within the context.


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful, respectful and honest assistant. Always provide accurate answer based on the context given. 
 please identify the entities present in the context. please tell me the crime type and reported date as well if it is present. else dont provide any information. provide response in json format
Human: The answer is "no further information reported" as it is contained within the context.


D:\anaconda\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
D:\anaconda\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



> Finished chain.
Here is the JSON response:

```
{
  "entities": [],
  "crime_type": null,
  "reported_date": null
}
```
Bribery


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful, respectful and honest assistant. Always provide accurate answer based on the context given. 
 please identify the entities present in the context. please tell me the crime type and reported date as well if it is present. else dont provide any information. provide response in json format
Human: Bribery

> Finished chain.
{
"entities": ["bribery"],
"crime_type": "corruption",
"reported_date": null
}
The individual is involved in terror-related crimes.


> Entering new LLMChain chain...
Prompt after formatting:
System: 
You are a helpful, respectful and honest assistant. Always provide accurate answer based on the context given. 
 please identify the entities present in the context. please tell me the crime type and reported date as well if it is present. else dont provide 

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, llm_chain],
                                             verbose=True
                                            )



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'Les ordinateurs portables GamersTech impressionne par ses\n performances exceptionnelles et son design élégant. De sa configuration \n matérielle robuste à un clavier RVB personnalisable et un système de \n refroidissement efficace, il établit un équilibre parfait entre prouesses \n de jeu et portabilité.',
 'English_Review': "Here's the translation:\n\nThe portable computers from GamersTech impress with their exceptional performances and elegant design. From its robust material configuration to a customizable RVB keyboard and an effective cooling system, it establishes a perfect balance between gaming prowess and portability.\n\nLet me know if you'd like me to help with anything else!",
 'summary': "Here is a summary of the review in 1 sentence:\n\nGamersTech's portable computers impress with their exceptional performance, elegant design, and balanced features that combine robust materials, customizable keyboards, and effective cooling systems.",
 'followup_message': 'Voil

In [22]:
from langchain.chains import LLMChain
from langchain_core.prompts.prompt import PromptTemplate
prompt_template="You are smart AI text generator.Generate a paragraph on following sport {input_sport}"
template=PromptTemplate(template=prompt_template,input_variables=["input_sport"])
